In [1]:
import rawpy
import cv2
from PIL import Image
import numpy as np
from multiprocessing.pool import Pool
import utils
import os


###############################################################################
# Instantiation
###############################################################################
lum_downsample = utils.Downsampling(ratio='4:4:4')
chr_downsample = utils.Downsampling(ratio='4:2:0')
image_block = utils.ImageBlock(block_height=8, block_width=8)
dct2d = utils.DCT2D(norm='ortho')
quantization = utils.Quantization()
zigzagScanning = utils.ZigzagScanning()
rle = utils.RLE()
entropy = utils.Entropy()
###############################################################################
# Preprocess
###############################################################################
# Read raw image file as array
# Generate random pixel values
raw = rawpy.imread(os.path.join('images', 'Nikon-D3500-Shotkit-4.NEF'))

    




In [2]:
rgb_img = raw.postprocess()


In [3]:
rgb_img

array([[[186, 148, 162],
        [189, 147, 163],
        [190, 150, 161],
        ...,
        [115,  94, 104],
        [120,  90, 105],
        [120,  87, 106]],

       [[185, 149, 162],
        [184, 151, 162],
        [183, 154, 160],
        ...,
        [111,  92, 104],
        [112,  92, 105],
        [112,  93, 105]],

       [[184, 152, 161],
        [180, 156, 160],
        [177, 154, 159],
        ...,
        [106,  91, 106],
        [104,  95, 104],
        [103,  99, 103]],

       ...,

       [[107,  85,  78],
        [110,  85,  78],
        [112,  84,  77],
        ...,
        [113,  76,  70],
        [115,  74,  70],
        [114,  80,  67]],

       [[100,  85,  79],
        [107,  85,  79],
        [113,  84,  80],
        ...,
        [113,  79,  66],
        [111,  78,  68],
        [111,  79,  69]],

       [[101,  83,  80],
        [107,  84,  80],
        [113,  83,  82],
        ...,
        [113,  80,  63],
        [111,  80,  67],
        [110,  79,  70]]

In [4]:
# Colorspace transform (RGB -> YCrCb)
ycc_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2YCrCb)


In [5]:
ycc_img

array([[[161, 146, 129],
        [161, 148, 129],
        [163, 147, 127],
        ...,
        [101, 138, 130],
        [101, 142, 130],
        [ 99, 143, 132]],

       [[161, 145, 129],
        [162, 144, 128],
        [163, 142, 126],
        ...,
        [ 99, 137, 131],
        [ 99, 137, 131],
        [100, 137, 131]],

       [[163, 143, 127],
        [164, 139, 126],
        [161, 139, 127],
        ...,
        [ 97, 134, 133],
        [ 99, 132, 131],
        [101, 129, 129]],

       ...,

       [[ 91, 139, 121],
        [ 92, 141, 120],
        [ 92, 142, 120],
        ...,
        [ 86, 147, 119],
        [ 86, 149, 119],
        [ 89, 146, 116]],

       [[ 89, 136, 122],
        [ 91, 139, 121],
        [ 92, 143, 121],
        ...,
        [ 88, 146, 116],
        [ 87, 145, 117],
        [ 87, 145, 118]],

       [[ 88, 137, 123],
        [ 90, 140, 122],
        [ 92, 143, 122],
        ...,
        [ 88, 146, 114],
        [ 88, 144, 116],
        [ 87, 144, 118]]

In [6]:
# Center
ycc_img = ycc_img.astype(int)-128

In [7]:
ycc_img

array([[[ 33,  18,   1],
        [ 33,  20,   1],
        [ 35,  19,  -1],
        ...,
        [-27,  10,   2],
        [-27,  14,   2],
        [-29,  15,   4]],

       [[ 33,  17,   1],
        [ 34,  16,   0],
        [ 35,  14,  -2],
        ...,
        [-29,   9,   3],
        [-29,   9,   3],
        [-28,   9,   3]],

       [[ 35,  15,  -1],
        [ 36,  11,  -2],
        [ 33,  11,  -1],
        ...,
        [-31,   6,   5],
        [-29,   4,   3],
        [-27,   1,   1]],

       ...,

       [[-37,  11,  -7],
        [-36,  13,  -8],
        [-36,  14,  -8],
        ...,
        [-42,  19,  -9],
        [-42,  21,  -9],
        [-39,  18, -12]],

       [[-39,   8,  -6],
        [-37,  11,  -7],
        [-36,  15,  -7],
        ...,
        [-40,  18, -12],
        [-41,  17, -11],
        [-41,  17, -10]],

       [[-40,   9,  -5],
        [-38,  12,  -6],
        [-36,  15,  -6],
        ...,
        [-40,  18, -14],
        [-40,  16, -12],
        [-41,  16, -10]]

In [8]:
# Downsampling
Y = lum_downsample(ycc_img[:,:,0])
Cr = chr_downsample(ycc_img[:,:,1])
Cb = chr_downsample(ycc_img[:,:,2])
ycc_img = np.stack((Y, Cr, Cb), axis=2)

In [9]:
Y

array([[ 33,  33,  35, ..., -27, -27, -29],
       [ 33,  34,  35, ..., -29, -29, -28],
       [ 35,  36,  33, ..., -31, -29, -27],
       ...,
       [-37, -36, -36, ..., -42, -42, -39],
       [-39, -37, -36, ..., -40, -41, -41],
       [-40, -38, -36, ..., -40, -40, -41]])

In [10]:
Y[0].size

6016

In [11]:
Cr[0].size

6016

In [12]:
Cb.size

24160256

In [13]:
ycc_img

array([[[ 33,  18,   1],
        [ 33,  18,   1],
        [ 35,  17,  -1],
        ...,
        [-27,   9,   3],
        [-27,  12,   3],
        [-29,  12,   3]],

       [[ 33,  18,   1],
        [ 34,  18,   1],
        [ 35,  17,  -1],
        ...,
        [-29,   9,   3],
        [-29,  12,   3],
        [-28,  12,   3]],

       [[ 35,  13,  -2],
        [ 36,  13,  -2],
        [ 33,  13,  -1],
        ...,
        [-31,   7,   5],
        [-29,   4,   2],
        [-27,   4,   2]],

       ...,

       [[-37,  14,  -7],
        [-36,  14,  -7],
        [-36,  13,  -8],
        ...,
        [-42,  18,  -9],
        [-42,  18, -11],
        [-39,  18, -11]],

       [[-39,  10,  -6],
        [-37,  10,  -6],
        [-36,  13,  -6],
        ...,
        [-40,  19, -12],
        [-41,  16, -11],
        [-41,  16, -11]],

       [[-40,  10,  -6],
        [-38,  10,  -6],
        [-36,  13,  -6],
        ...,
        [-40,  19, -12],
        [-40,  16, -11],
        [-41,  16, -11]]

In [14]:
# Create 8x8 blocks
blocks, indices = image_block.forward(ycc_img)

In [15]:
blocks.size

72480768

In [16]:
indices.size

3397536

In [ ]:
toEncode_Y = []
toEncode_CbCr = []
def process_block(block, index):
    
    #Prediction  -> Prediction error
    
    # DCT
    encoded = dct2d.forward(block)
    if index[2] == 0:
        channel_type = 'lum'
    else:
        channel_type = 'chr'
        
    # Quantization
    encoded = quantization.forward(encoded, channel_type)
    
    # RLE + zigzag scanning
    encoded = zigzagScanning.forward(encoded)
    encoded = rle.forward(encoded)
    
    assert channel_type in ('lum', 'chr')
        
    if channel_type == 'lum':
        toEncode_Y.append(encoded)
    else:
        toEncode_CbCr.append(encoded)

    # Reverse Entropy coding (Arithmetic)
    #decoded = entropy.backward(encoded, prob)
    decoded = rle.backward(encoded)
    # Reverse RLE + zigzag scanning
    decoded = zigzagScanning.backward(decoded)
    # Dequantization
    decoded = quantization.backward(decoded, channel_type)
    
    # Reverse DCT
    compressed = dct2d.backward(decoded)
    return compressed,

    #Reverse Prediction 

In [52]:
compressed = []
for i in range(0,int(blocks.size/64)-1):
  compressed.append(process_block(blocks[i],indices[i]))
  
encodedY, codebook = entropy.huffman_encoding(toEncode_Y)
encodedCbCr, codebook= entropy.huffman_encoding(toEncode_CbCr)

KeyboardInterrupt: 

In [37]:
ycc_img_compressed = image_block.backward(compressed, indices)

In [38]:
ycc_img_compressed = (ycc_img_compressed+128).astype('uint8')

In [39]:
rgb_img_compressed = cv2.cvtColor(ycc_img_compressed, cv2.COLOR_YCrCb2RGB)

In [44]:
Image.fromarray(rgb_img_compressed).save(os.path.join('images', 'result.bmp'))

In [69]:
indices[78]

array([  0, 208,   0])

In [54]:
encoded = dct2d.forward(blocks[45])
encoded

array([[ 2.90875000e+02, -1.69478363e+00, -2.73681813e+00,
        -5.10360788e-01,  1.25000000e-01,  7.47819060e-01,
        -1.70765234e+00, -4.95486606e-01],
       [-1.91283896e+00,  2.78086824e+00, -2.77867035e+00,
        -2.05006442e+00, -6.21193306e-01,  1.30527032e+00,
        -2.32448968e-01, -2.78484139e-01],
       [-3.63748402e+00, -1.22562078e+00, -3.98115530e+00,
        -4.30722302e+00,  2.87211716e+00,  2.31084434e-01,
         2.16161165e+00,  5.88808898e-01],
       [-2.88624110e+00,  7.19638354e-01,  6.51968137e-01,
        -1.72001784e+00, -1.09145256e-01,  1.90909753e+00,
        -2.62688272e-02,  4.75861606e-01],
       [ 6.25000000e-01, -8.20471429e-01, -6.41674661e-01,
         2.46860233e+00,  1.37500000e+00, -7.15009036e-01,
        -1.22249893e+00,  1.07661100e+00],
       [-1.07366907e+00,  3.47361699e-01,  9.09840038e-01,
        -1.33795685e-01,  2.01950297e-01,  1.30580428e+00,
         2.74192253e-01, -8.71351903e-01],
       [-3.22877066e+00, -6.081202

In [70]:
for block in enumerate(0,blocks):
        if indices[block] == 0:
                channel_type = 'lum'
                print(block)
                break
        else:
                channel_type = 'chr'
        index    
    # Quantization
encoded = quantization.forward(encoded, channel_type)
encoded

TypeError: only integer scalar arrays can be converted to a scalar index

In [56]:
encoded = zigzagScanning.forward(encoded)
encoded

[18.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.0,
 0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 -0.0,
 -0.0,
 -0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.0,
 0.0,
 -0.0]

In [57]:
encoded = rle.forward(encoded)
encoded

[(18.0, 1), (-0.0, 63)]

In [66]:

encoded, prob = entropy.forward(encoded)
encoded

TypeError: 'int' object is not iterable

In [58]:
decoded = rle.backward(encoded)
decoded

[18.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0,
 -0.0]

In [59]:
decoded = zigzagScanning.backward(decoded)
decoded

[[18.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0],
 [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]]

In [60]:
decoded = quantization.backward(decoded, channel_type)
decoded

array([[288.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
       [ -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
       [ -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
       [ -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
       [ -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
       [ -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
       [ -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.],
       [ -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.,  -0.]])

In [61]:
compressed = dct2d.backward(decoded)
compressed

array([[36., 36., 36., 36., 36., 36., 36., 36.],
       [36., 36., 36., 36., 36., 36., 36., 36.],
       [36., 36., 36., 36., 36., 36., 36., 36.],
       [36., 36., 36., 36., 36., 36., 36., 36.],
       [36., 36., 36., 36., 36., 36., 36., 36.],
       [36., 36., 36., 36., 36., 36., 36., 36.],
       [36., 36., 36., 36., 36., 36., 36., 36.],
       [36., 36., 36., 36., 36., 36., 36., 36.]])

In [47]:


    
    # RLE + zigzag scanning

encoded1, prob = entropy.forward(encoded)
encoded_array.append(encoded1)
decoded = rle.backward(encoded)
    # Reverse RLE + zigzag scanning
decoded = zigzagScanning.backward(decoded)
    # Dequantization
decoded = quantization.backward(decoded, channel_type)
    
    # Reverse DCT
compressed = dct2d.backward(decoded)


In [48]:
encoded_array

[1]